In [1]:
import crocoddyl
import numpy 
import random
from pendulum import *
from cartpole_utils import *

In [2]:
model = cartpole_model()
# x , angle(0-> up, -360 -> down)
x0 = np.matrix([ 0, 0, 0, -0.99 ]).T
T  = 50
problem = crocoddyl.ShootingProblem(x0, [ model ]*T, model)

In [3]:
ddp = crocoddyl.SolverDDP(problem)
ddp.solve()

True

In [4]:
ddp.xs[1]


matrix([[ 2.67285790e-12],
        [-4.94999995e-04],
        [ 5.34571580e-09],
        [-9.89999989e-01]])

In [5]:
%%capture
%matplotlib inline
from IPython.display import HTML

# Create animation
anim = animateCartpole(ddp.xs)


In [6]:
HTML(anim.to_html5_video())

In [7]:
import example_robot_data


robot = example_robot_data.loadDoublePendulum()
robot_model = robot.model

state = crocoddyl.StateMultibody(robot_model)
actModel = ActuationModelDoublePendulum(state, actLink=1)

weights = np.array([1, 1, 1, 1] + [0.1] * 2)
runningCostModel = crocoddyl.CostModelSum(state, actModel.nu)
terminalCostModel = crocoddyl.CostModelSum(state, actModel.nu)
xRegCost = crocoddyl.CostModelState(state, crocoddyl.ActivationModelQuad(state.ndx), state.zero(), actModel.nu)
uRegCost = crocoddyl.CostModelControl(state, crocoddyl.ActivationModelQuad(1), actModel.nu)
xPendCost = CostModelDoublePendulum(state, crocoddyl.ActivationModelWeightedQuad(np.matrix(weights).T), actModel.nu)

dt = 1e-2

runningCostModel.addCost("uReg", uRegCost, 1e-4 / dt)
runningCostModel.addCost("xGoal", xPendCost, 1e-5 / dt)
terminalCostModel.addCost("xGoal", xPendCost, 1e4)

runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actModel, runningCostModel), dt)
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actModel, terminalCostModel), dt)

# Creating the shooting problem and the FDDP solver
T = 100
x0 = [3.14, 0, 0., 0.]
problem = crocoddyl.ShootingProblem(np.matrix(x0).T, [runningModel] * T, terminalModel)
fddp = crocoddyl.SolverDDP(problem)


NameError: name 'ActuationModelDoublePendulum' is not defined